# Implementation of a Stark proof on a Lucas sequence computation

#### Amine Abdeljaoued - CSE303 under the supervision of Sarah Bordage

In [58]:
import random
from hashlib import sha256
from Merkle import *

We will define our Lucas sequence $U(P,Q)$ as follow:<br> 
$$ \begin{equation}
    \begin{cases}
        U_0(P,Q)=0\\
        U_1(P,Q)=1\\
        U_n(P,Q)=PU_{n-1}(P,Q) - QU_{n-2}(P,Q) \ \  \forall n \ge 2
    \end{cases}\,.
\end{equation}$$

We will work with the prime number $3221225473 = 3 \cdot 2^{30} + 1$ and we denote  $\mathbb{F}=\mathbb{F}_{3221225473}$ the finite field of size $p$.

We will produce a STARK proof attesting of the following statement: <br> **I know field elements P,Q $\in \mathbb{F}$ such that the 15th element of the sequence ($U_{14}$) with parameters P and Q is 409593865**

The channel implemented in the following cell will receive the MerkleTree root from the prover in order to verify the latter's merkle proofs.

In [59]:
channel = []

## Trace and Polynomial

In [60]:
F = GF(3221225473)

#### We create the trace

The "trace" is the computational trace i.e the sequence elements up to the one we are interested in.

In [61]:
a = [0,1]
P = 5
Q = 2
while len(a)<15:
    a.append(P*a[-1] - Q*a[-2])

#### We define our polynomial

We start by defining its domain, and then use a Lagrange interpolation to get the polynomial with domain G and the trace a as its co-domain.
We want a generator of a subgroup of $\mathbb{F}^\times$ of order 16 and we know it exists since 16 divides $3\cdot 2^{30}$, the size of our group $\mathbb{F}$.

We use the following given on the STARK tutorial: *Hint: When $k$ divides $|\mathbb{F}^\times|$, $g^k$ generates a group of size $\frac {|\mathbb{F}^\times|}{k}$, and the n-th power of some field element $x$ can be computed by calling `x ** n `.*

In [62]:
g = F.multiplicative_generator() ** (3 * 2**26)

G = [g**i for i in range(16)] #Domain of our polynomial

In [63]:
R.<X> = PolynomialRing(F)

In [64]:
points = [(G[i],a[i]) for i in range(15)]
f=R.lagrange_polynomial(points)

In [65]:
print("Interpolating polynomial f(X) =", f)

Interpolating polynomial f(X) = 1591378048*X^14 + 1480639541*X^13 + 1293339165*X^12 + 2205269950*X^11 + 807315743*X^10 + 2715312626*X^9 + 1979814329*X^8 + 1074125415*X^7 + 261527103*X^6 + 1438334297*X^5 + 671075275*X^4 + 3065900240*X^3 + 1361401121*X^2 + 2515909827*X + 87235631


#### Evaluation of our polynomial on a larger coset

We will choose a domain G1 16 times larges than our initial one, therefore of size 256. We then create a coset of G1.

In [66]:
g1 = F.multiplicative_generator()
g2 = g1**(3 * 2**22)

G1 = [g2**i for i in range(256)]

eval_domain = [g1 * h for h in G1]

## Constraints

#### We firstly define our constraints

On a, our constraints are:

1) a[0]=0

2) a[1]=1

3) a[14] = 409593865

4) $\forall i  < 13$, $a[i+2]= P \cdot a[i+1] - Q\cdot a[i]$

Translated for our polynomial f, it gives:

1) $f(x)$ evaluates to 0 for $x=g^0=1$

2) $f(x) - 1 $ evaluates to 0 for $x=g^1$

3) $f(x) - 409593865$ evaluates to 0 for $x=g^{14}$

4) $f(g^2 \cdot x) - P\cdot f(g \cdot x) + Q\cdot f(x)$ evaluates to 0 for $x \in G \backslash \{g^{15},g^{14},g^{13}\}$

Written in terms of rational functions, this gives:

1) $p_1(x)=\frac{f(x)}{x-1}$ is a polynomial

2) $p_2(x)=\frac{f(x)-1}{x-g}$ is a polynomial

3) $p_3(x)=\frac{f(x)-409593865}{x-g^{14}}$ is a polynomial

4) $$p_4(x)=\frac{f(g^2 \cdot x) - P\cdot f(g \cdot x) + Q\cdot f(x)}{\prod_{i=0}^{12} (x-g^i)}= \frac{f(g^2 \cdot x) - P\cdot f(g \cdot x) + Q\cdot f(x)}{\frac{x^{16}-1}{(x-g^{15})(x-g^{14})(x-g^{13})}}$$ is a polynomial

#### We will now define the constraints using sagemath univariate polynomial ring's operations

In [67]:
p1 = f/ (X-1)
p2 = (f-1)/ (X-g)
p3 = (f - 409593865)/ (X - g**14)
p4 = ( f(g**2 * X) - P*f(g*X) + Q*f(X))

In [68]:
print("p1(X) =", p1)
print("p2(X) =", p2)
print("p3(X) =", p3)
print("p4(X) =", p4)

p1(X) = 1591378048*X^13 + 3072017589*X^12 + 1144131281*X^11 + 128175758*X^10 + 935491501*X^9 + 429578654*X^8 + 2409392983*X^7 + 262292925*X^6 + 523820028*X^5 + 1962154325*X^4 + 2633229600*X^3 + 2477904367*X^2 + 618080015*X + 3133989842
p2(X) = 1591378048*X^13 + 1494727035*X^12 + 357395372*X^11 + 1850590412*X^10 + 183130280*X^9 + 800191002*X^8 + 1117252255*X^7 + 1951935049*X^6 + 60573156*X^5 + 1568431439*X^4 + 537230794*X^3 + 816838766*X^2 + 2504659179*X + 1876245401
p3(X) = 1591378048*X^13 + 720129134*X^12 + 1459058*X^11 + 2911415979*X^10 + 1751087406*X^9 + 2975797655*X^8 + 658235586*X^7 + 3190440437*X^6 + 971519052*X^5 + 2988093240*X^4 + 853232600*X^3 + 1926404082*X^2 + 2607091556*X + 2915649450
p4(X) = 1772948010*X^14 + 2863677125*X^13 + 84962123*X^12 + 2167507865*X^11 + 2823498213*X^10 + 2425853534*X^9 + 2953612740*X^8 + 317852993*X^7 + 90808569*X^6 + 1900351788*X^5 + 357121872*X^4 + 735864615*X^3 + 1755196154*X^2 + 2473793972*X + 3046754211


#### We then construct the composition polynomial

In [69]:
#The Verifier provides the random values
a1 = random.randrange(0,3221225473,1)
a2 = random.randrange(0,3221225473,1)
a3 = random.randrange(0,3221225473,1)
a4 = random.randrange(0,3221225473,1)

In [97]:
p = a1 * p1 + a2 * p2 + a3 * p3 + a4 * p4

Then we commit the root of the merkle tree of the evaluation of this composition polynomial

In [98]:
tree = MerkleTree([p(x) for x in eval_domain])
channel.append(tree.root)

#### Check that this polynomial is close to a polynomial of low degree

We need to prove that p is close to a polynomial of degree<16
For that matter we will create p_interp, the polynomial resulting from the interpolation of 16 random points on eval_domain and their image. 

In [99]:
#The Verifier asks for the p(x)s, provided by the prover
rand_points=random.choices(eval_domain,k=16)
evals = [p(x) for x in rand_points]

###### The Prover now needs to prove that each point of evals is correct by providing a merkle proof, further verified by the verifier.

In [100]:
def merkle_proof(tree, paths, point):
    right_path = None
    for path in paths:
        if path[-1]==point:
            right_path = path
    if right_path == None:
        return -1
    
    proof = []
    path = []
    z = tree.root
    for i in range(1,len(right_path)):
        if z.right.label==right_path[i]:
            proof.append(z.left)
            path.append(z.right)
            z = z.right
        
        elif z.left.label==right_path[i] : 
            proof.append(z.right)
            path.append(z.left)
            z = z.left
        
        else:
            if z==proof[-1].right:
                path.append(proof[-1].left)
            else:
                path.append(proof[-1].right)
            
            proof.append(z)
            
        
        
    return path,proof

##### The verifier can now verify that a point of evals is correct according the the previous root commit using the merkle proof emitted.

In [101]:
def verify_proof(root,right_path,proof):
    '''Iteratively ssa the sum of element of right_path 
    and element of proof to see if it gives next element of
    right_path until we get the root and compare with the root
    sent previously by the prover.'''
    #We need to work in reverse
    n = len(proof)
    for i in range(n-1,-1,-1):
        hashed1 = sha256( (  str(proof[i].value) + str(right_path[i].value) ).encode()).hexdigest()
        hashed2 = sha256( (  str(right_path[i].value) + str(proof[i].value) ).encode()).hexdigest()

        if i==0:
            if hashed1!=root.value and hashed2 != root.value:
                print("Merkle proof not correct")
                return False
        else:
            if hashed1!=right_path[i-1].value and hashed2 != right_path[i-1].value:
                print("Merkle proof not correct")
                return False
    
    return True
    

In [102]:
paths = tree.get_paths()

for point in evals:
    point = str(point)
    right_path,proof= merkle_proof(tree,paths,point)
    validity = verify_proof(tree.root,right_path,proof)
    assert validity==True
    

The verifier has now checked the validity of these points and we can proceed to create p_interp. The following cell might fail with a ZeroDivisionError.

In [103]:
points = [(rand_points[i],evals[i]) for i in range(16)]
p_interp = R.lagrange_polynomial(points)

#### Now, we compare p_interp to p taking a random point. Having an equality on this random point implies wiht high probability that our composition polynomial was indeed a polynomial of low degree and consequently that the computation is correct.

For further details on why this is true for a mathematical point of view, see report.

In [105]:
random_point = random.choice(eval_domain)
print(p_interp(random_point)==p(random_point))

True
